In [13]:
pip install mysql-connector-python

In [14]:
import mysql.connector
from getpass import getpass

In [15]:
password = getpass()

In [31]:
#1. Establish a connection between Python and the Sakila database.
conn = mysql.connector.connect(host = "localhost",
                               user = "root",
                               password = password)

In [32]:
#cheking if connection is successfull
conn.is_connected()

True

In [33]:
#now are creating a cursor, that will hole our query, execute it, and fetch my results -object that go from python to sql
cursor = conn.cursor()

In [52]:
#2. Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:
##engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
##month: an integer representing the month for which rental data is to be retrieved.
##year: an integer representing the year for which rental data is to be retrieved.
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine #import library
def rentals_month(engine, month, year):
    query = f"""
        SELECT * 
        FROM rental 
        WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """

    rental_data = pd.read_sql_query(query, engine)
    
    return rental_data


In [53]:
rental_data

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id
0,11496,2006-02-14 15:16:03,2047,155,None,1
1,11541,2006-02-14 15:16:03,2026,335,None,1
2,11563,2006-02-14 15:16:03,1545,83,None,1
3,11577,2006-02-14 15:16:03,4106,219,None,2
4,11593,2006-02-14 15:16:03,817,99,None,1
...,...,...,...,...,...,...
177,15862,2006-02-14 15:16:03,925,215,None,1
178,15867,2006-02-14 15:16:03,837,505,None,2
179,15875,2006-02-14 15:16:03,3611,41,None,1
180,15894,2006-02-14 15:16:03,4416,168,None,1


In [66]:
#3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.
def rental_count_month(rental_data, month, year):

    rental_count = rental_data.groupby('customer_id').size().reset_index(name=f'rentals_{month}_{year}')
    
    return rental_count


result_df = rental_count_month(rental_data, 5, 2024)
result_df_1 = pd.DataFrame(result_df)
# Print the resulting DataFrame
result_df_1

,customer_id,rentals_5_2024
0,5,1
1,9,1
2,11,1
3,14,1
4,15,2
...,...,...
153,587,1
154,590,1
155,592,1
156,596,1


In [68]:
#4. 
def compare_rentals(df1, df2):
    final_df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    
    # Calculate the difference between rentals in the two months
    final_df['difference'] = final_df.iloc[:, 1] - final_df.iloc[:, 2]
    
    return final_df

In [69]:
df1 = pd.DataFrame({'customer_id': [1, 2, 3], 'rentals_month1': [5, 7, 3]})
df2 = pd.DataFrame({'customer_id': [1, 2, 4], 'rentals_month2': [3, 6, 2]})

# Call the compare_rentals function
result_df = compare_rentals(df1, df2)

# Print the resulting DataFrame
print(result_df)

   customer_id  rentals_month1  rentals_month2  difference
0            1             5.0             3.0         2.0
1            2             7.0             6.0         1.0
2            3             3.0             0.0         3.0
3            4             0.0             2.0        -2.0


In [71]:
result_df_final = pd.DataFrame(result_df)
result_df_final 

,customer_id,rentals_month1,rentals_month2,difference
0,1,5.0,3.0,2.0
1,2,7.0,6.0,1.0
2,3,3.0,0.0,3.0
3,4,0.0,2.0,-2.0
